## Code to compile data from multiple source regarding pulsating aurora and PFISR and write to dictionary

written by Riley Troyer Fall 2021

In [9]:
# Start by defining the directories where the data is stored
proj_dir = '/media/sf_semeter-inversion/LAMP_Data_saved/'
# pfisr_dir = proj_dir + 'data/pfisr-data/barker-code/'
pfisr_dir = '/media/sf_semeter-inversion/LAMP_Data/'

In [10]:
# Libraries
#import rtroyer_useful_functions as rt_func

from datetime import datetime as dt
from datetime import time as dt_time
from datetime import timedelta
import datetime
import h5py
import numpy as np
import os
import pickle
import pytz

In [16]:
def get_isr_data(pfisr_filename, pfisr_data_dir):
    """Function to get relevant data from PFISR datafile.
    INPUT
    pfisr_filename
        type: str
        about: data file name, should be .h5 file
    pfisr_data_dir
        type: str
        about: directory where isr data is stored
    OUTPUT
    utc_time
        type: array of datetimes
        about: time stamp for the start of each measurement
    unix_time
        type: array of floats
        about: unix timestamp for start of each measurement
    pfisr_altitude
        type: array of float
        about: altitude stamp for each measurement in meters
    e_density
        type: array of float
        about: electron number density in m^-3
    de_density
        type: array of float
        about: error in number density
    """
    
    # Read in the h5 file
    pfisr_file = h5py.File(pfisr_data_dir + pfisr_filename, 'r')

    # Get the different beams and select specified angle
    beam_angle = 90
    beams = np.array(pfisr_file['BeamCodes'])

    # Get the beam with a 90 degree elevation angle
    indexes = np.linspace(0, len(beams)-1, len(beams))
    beam_num = int(indexes[np.abs(beams[:,2] - beam_angle) == 0][0])

    # Get time and convert to utc datetime
    unix_time = np.array(pfisr_file['Time']['UnixTime'])[:,0]
    utc_time = np.array([datetime.datetime.utcfromtimestamp(d) 
                         for d in unix_time])

    # Get the altitude array
    pfisr_altitude = np.array(pfisr_file['NeFromPower']
                              ['Altitude'])[beam_num, :]

    # Get the uncorrected number density array
    e_density = np.array(pfisr_file['NeFromPower']
                         ['Ne_NoTr'])[:, beam_num, :]

    # Take the transpose
    e_density = np.transpose(e_density)
    
    # Find the noise floor by averaging between 55km and 60km
    #...assume this should be zero
    
    # Calculate the power given that power = density/range^2
    pfisr_range = np.array(pfisr_file['NeFromPower']
                           ['Range'])[0, :]

    # Turn 1D array into 2D array for elementwise division
    pfisr_range = np.array([pfisr_range,]*e_density.shape[1])
    pfisr_range = np.transpose(pfisr_range)
    pfisr_power = np.divide(e_density, pfisr_range**2)

    # Get the power bias
    noise_floor = np.nanmean(pfisr_power[(pfisr_altitude > 55000)
                                    & (pfisr_altitude < 60000), :],
                              axis=0)

    # Loop through each column and subtract off noise floor
    for j in range(pfisr_power.shape[1]):
        pfisr_power[:, j] = pfisr_power[:, j] - noise_floor[j]   

    # Calculate new unbiased density
    e_density = np.multiply(pfisr_power, pfisr_range**2)
        
    
    # Get error values
    try:
        de_density = np.array(pfisr_file['NeFromPower']
                              ['errNe_NoTr'])[:, beam_num, :]
        de_density = np.transpose(de_density)
    except:
        de_density = np.array(pfisr_file['NeFromPower']
                              ['dNeFrac'])[:, beam_num, :]
        de_density = np.transpose(de_density)
        de_density = de_density * e_density

    # Close file
    pfisr_file.close()
    
    return utc_time, unix_time, pfisr_altitude, e_density, de_density

In [17]:
# Loop through each pa date and find time from nearest substorm
pa_pfisr_dict = {}

auroral_type = 'pa'
low_alt_cutoff = 60
cutoff = 12

# Define alaska timezone
alaska_tzinfo = pytz.timezone("US/Alaska")
mlt_midnight = dt_time(10, 54, 0)

# Get the date and start time of measurements
date = '2022-03-05'
#start_time = date + ' ' + '10:00:00'
#end_time = date + ' ' + '20:00:00'

start_time = date + ' ' + '08:00:00'
end_time = date + ' ' + '16:00:00'

# Convert to datetime
start_time = dt.strptime(start_time, '%Y-%m-%d %H:%M:%S')
end_time = dt.strptime(end_time, '%Y-%m-%d %H:%M:%S')
date = start_time.date()

# Read in the PFISR data file
date_str = (str(date.year).zfill(4) 
            + str(date.month).zfill(2)
            + str(date.day).zfill(2))
pfisr_files = os.listdir(pfisr_dir)

# Take only h5 files
pfisr_files = [f for f in pfisr_files if f.endswith('.h5')]

# In case there are multiple files just take first one
pfisr_filename = [f for f in pfisr_files if date_str in f][0]
print(pfisr_filename) 

# (pfisr_time,
#  unix_time,
#  pfisr_alt,
#  e_density,
#  de_density) = rt_func.get_isr_data(pfisr_filename, pfisr_dir)
(pfisr_time,
 unix_time,
 pfisr_alt,
 e_density,
 de_density) = get_isr_data(pfisr_filename, pfisr_dir)
#print(e_density)

# Select data over specified altitude range
low_alt_cutoff = 60000

# Select only between 60km and 140km
altitude_slice = (pfisr_alt 
                  < 140000) & (pfisr_alt 
                               > low_alt_cutoff)
e_density = e_density[altitude_slice, :]
de_density = de_density[altitude_slice, :]
pfisr_alt = pfisr_alt[altitude_slice]

# Filter to only between pulsating aurora times
e_density = e_density[:, (pfisr_time >= start_time)
                      & (pfisr_time <= end_time)]
de_density = de_density[:, (pfisr_time >= start_time)
                        & (pfisr_time <= end_time)]
pfisr_time = pfisr_time[(pfisr_time >= start_time)
                        & (pfisr_time <= end_time)]
print(e_density)
print(pfisr_time)

20220305.001_bc_nenotr_1min.h5
[[-3.00907315e+09 -1.26200046e+09  3.40608614e+07 ... -8.06954603e+08
  -9.60211892e+08  5.47959060e+08]
 [-2.29227398e+09 -2.02374758e+09 -4.18927849e+08 ...  1.76533808e+08
  -1.22626080e+09 -1.65608009e+08]
 [-8.56517488e+08 -7.86200601e+08  1.10087757e+09 ... -1.12984456e+09
  -1.60069352e+09 -2.31929293e+08]
 ...
 [-3.45248787e+08  1.94132345e+09  1.88209512e+10 ...  3.86343767e+10
   4.16396219e+10  4.69654981e+10]
 [-3.38201227e+08  1.32457443e+10 -4.26803284e+09 ...  4.64801524e+10
   3.73881236e+10  5.11298735e+10]
 [ 2.38677315e+09  9.01145847e+09  1.06573126e+10 ...  3.48509977e+10
   3.93261584e+10  3.43943561e+10]]
[datetime.datetime(2022, 3, 5, 8, 0, 59)
 datetime.datetime(2022, 3, 5, 8, 2, 2)
 datetime.datetime(2022, 3, 5, 8, 3, 5)
 datetime.datetime(2022, 3, 5, 8, 4, 9)
 datetime.datetime(2022, 3, 5, 8, 5, 12)
 datetime.datetime(2022, 3, 5, 8, 6, 16)
 datetime.datetime(2022, 3, 5, 8, 7, 19)
 datetime.datetime(2022, 3, 5, 8, 8, 22)
 datetim

In [18]:
# Loop through each time and select lowest altitude
for n in range(0, len(pfisr_time)):

    # Select profile for time
    density_profile = e_density[:, n]

    # Also get the profile for the error in this
    error_profile = de_density[:, n]

    # And make sure all of the dates are the same
    profile_time = pfisr_time[n]

    # Get local time and magnetic local time
    ak_time = profile_time.astimezone(alaska_tzinfo)
    ak_time = dt.combine(ak_time.date(), ak_time.time()) 
    mlt_delay = profile_time - dt.combine(profile_time.date(),
                                          mlt_midnight)
    mlt_time = dt.combine(profile_time.date(), 
                          dt_time(0, 0, 0)) + mlt_delay

    # Select only densities > 10^10
    density_slice = density_profile >= 1e10

    # Select only data points with reasonable error
    error_slice = error_profile < 5e9

    # Select only only data points with continuity with previous
    # Select only densities that connect to points above
    continuity_slice = np.zeros(len(pfisr_alt), dtype=bool)

    for j in np.arange(len(pfisr_alt)):

        if j < (len(pfisr_alt) - 20):

            if ((np.min(density_profile[j+1:j+20]) > 0)
                & (np.median(density_profile[j-2:j+2]) 
                   < np.median(density_profile[j+1:j+20]))):
                continuity_slice[j] = True

            else:
                continuity_slice[j] = False

        else: 
            continuity_slice[j] = False

    density_profile = density_profile[density_slice 
                                      & error_slice
                                      & continuity_slice]

    #print(density_profile)
    # Account for possibly no density above threshold
    try:
        # Get lowest altitude value
        lowest_alt = pfisr_alt[density_slice
                               & error_slice
                               & continuity_slice][0]

    except:
        lowest_alt = np.nan

    # Write to dictionary
    pa_pfisr_dict[profile_time] = {'altitude' : lowest_alt,
                                  'local_time' : ak_time,
                                  'mlt_time' : mlt_time}
    print(lowest_alt,ak_time,mlt_time)
print('Finished.')

111488.703125 2022-03-05 05:00:59 2022-03-04 21:06:59
110739.21875 2022-03-05 05:02:02 2022-03-04 21:08:02
109240.2578125 2022-03-05 05:03:05 2022-03-04 21:09:05
108490.7734375 2022-03-05 05:04:09 2022-03-04 21:10:09
109989.734375 2022-03-05 05:05:12 2022-03-04 21:11:12
110739.21875 2022-03-05 05:06:16 2022-03-04 21:12:16
110739.21875 2022-03-05 05:07:19 2022-03-04 21:13:19
109989.734375 2022-03-05 05:08:22 2022-03-04 21:14:22
109989.734375 2022-03-05 05:09:26 2022-03-04 21:15:26
112238.1796875 2022-03-05 05:10:29 2022-03-04 21:16:29
112238.1796875 2022-03-05 05:11:32 2022-03-04 21:17:32
108490.7734375 2022-03-05 05:12:36 2022-03-04 21:18:36
108490.7734375 2022-03-05 05:13:39 2022-03-04 21:19:39
108490.7734375 2022-03-05 05:14:42 2022-03-04 21:20:42
108490.7734375 2022-03-05 05:15:46 2022-03-04 21:21:46
108490.7734375 2022-03-05 05:16:49 2022-03-04 21:22:49
108490.7734375 2022-03-05 05:17:52 2022-03-04 21:23:52
106242.3359375 2022-03-05 05:18:56 2022-03-04 21:24:56
109240.2578125 2022-

79261.0078125 2022-03-05 09:02:18 2022-03-05 01:08:18
81509.453125 2022-03-05 09:03:21 2022-03-05 01:09:21
82258.9375 2022-03-05 09:04:24 2022-03-05 01:10:24
81509.453125 2022-03-05 09:05:28 2022-03-05 01:11:28
79261.0078125 2022-03-05 09:06:31 2022-03-05 01:12:31
81509.453125 2022-03-05 09:07:34 2022-03-05 01:13:34
83008.4140625 2022-03-05 09:08:38 2022-03-05 01:14:38
82258.9375 2022-03-05 09:09:41 2022-03-05 01:15:41
82258.9375 2022-03-05 09:10:44 2022-03-05 01:16:44
80759.9765625 2022-03-05 09:11:48 2022-03-05 01:17:48
81509.453125 2022-03-05 09:12:51 2022-03-05 01:18:51
83757.8984375 2022-03-05 09:13:55 2022-03-05 01:19:55
84507.3828125 2022-03-05 09:14:58 2022-03-05 01:20:58
84507.3828125 2022-03-05 09:16:01 2022-03-05 01:22:01
89004.265625 2022-03-05 09:17:05 2022-03-05 01:23:05
92002.1875 2022-03-05 09:18:08 2022-03-05 01:24:08
95000.1171875 2022-03-05 09:19:11 2022-03-05 01:25:11
92751.671875 2022-03-05 09:20:15 2022-03-05 01:26:15
88254.78125 2022-03-05 09:21:18 2022-03-05 01:

79261.0078125 2022-03-05 11:49:08 2022-03-05 03:55:08
80010.4921875 2022-03-05 11:50:12 2022-03-05 03:56:12
80010.4921875 2022-03-05 11:51:15 2022-03-05 03:57:15
79261.0078125 2022-03-05 11:52:19 2022-03-05 03:58:19
81509.453125 2022-03-05 11:53:22 2022-03-05 03:59:22
81509.453125 2022-03-05 11:54:25 2022-03-05 04:00:25
83008.4140625 2022-03-05 11:55:29 2022-03-05 04:01:29
81509.453125 2022-03-05 11:56:32 2022-03-05 04:02:32
82258.9375 2022-03-05 11:57:35 2022-03-05 04:03:35
82258.9375 2022-03-05 11:58:39 2022-03-05 04:04:39
80010.4921875 2022-03-05 11:59:42 2022-03-05 04:05:42
82258.9375 2022-03-05 12:00:45 2022-03-05 04:06:45
81509.453125 2022-03-05 12:01:49 2022-03-05 04:07:49
81509.453125 2022-03-05 12:02:52 2022-03-05 04:08:52
81509.453125 2022-03-05 12:03:55 2022-03-05 04:09:55
82258.9375 2022-03-05 12:04:59 2022-03-05 04:10:59
80759.9765625 2022-03-05 12:06:02 2022-03-05 04:12:02
80759.9765625 2022-03-05 12:07:06 2022-03-05 04:13:06
80010.4921875 2022-03-05 12:08:09 2022-03-05 0

In [19]:
# # Save dictionary to file
# with open(proj_dir + 'data/pfisr-data/statistics/'
#           'pa-pfisr-data-dict-v20220304.pickle',
#                   'wb') as handle:
#     pickle.dump(pa_pfisr_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(proj_dir+
          'pa-pfisr-data-dict-v20220305.pickle',
                  'wb') as handle:
    pickle.dump(pa_pfisr_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)